# Programming a simple ODE solver

Ordinary differential equations (ODEs) are widely used in science and engineering, in particular for modeling dynamic
processes. While simple ODEs can be solved with analytical methods, non-linear ODEs are generally not possible
to solve in this way, and we need to apply numerical methods. In this chapter we will see how we can program
general numerical solvers that can be applied to any ODE. We will first consider scalar ODEs, i.e., ODEs with a single
equation and a single unknown, and in Section 1.4 we will
extend the ideas to systems of coupled ODEs. Understanding the concepts
of this chapter is useful not only for programming your own ODE solvers, but also for using a wide variety of
general-purpose ODE solvers available both in Python and other programming languages.


# Creating a general-purpose ODE solver
<div id="sec:fe_intro"></div>
When solving ODEs analytically one will typically consider a specific ODE or a class of ODEs, and try to derive
a formula for the solution. In this chapter we want to implement numerical solvers that can be applied to any ODE,
not restricted to a single example or a particular class of equations. For this purpose, we need a general abstract
notation for an arbitrary ODE. We will write the ODEs on the following form:

<!-- Equation labels as ordinary links -->
<div id="ode0"></div>

$$
\begin{equation}
u^{\prime}(t) = f(t,u(t)),
\label{ode0} \tag{1}
\end{equation}
$$

which means that the ODE is fully specified by the definition of the right-hand side function $f(t,u)$. Examples of this
function may be:

$$
\begin{align*}
f(t,u) &= \alpha u,\quad\hbox{exponential growth}\\
f(t,u) &= \alpha u\left(  1-\frac{u}{R}\right),\quad\hbox{logistic growth}\\
f(t,u) &= -b|u|u + g,\quad\hbox{falling body in a fluid}
\end{align*}
$$

Notice that, for generality, we write all these right-hand sides as functions of both $t$ and $u$, although the
mathematical formulations only involve $u$. This general formulation is not strictly needed in the
mathematical equations, but it is very convenient when we start programming, and want to use the same solver
for a wide range of ODE models. We will discuss this in more detail later. 
Our aim is now to write functions and classes that take $f$ as input, and solve the corresponding ODE to produce
$u$ as output.

In order for ([1](#ode0)) to have a unique solution we need to specify the *initial condition* for $u$, which is 
the value of the solution at time $t=t_0$. The resulting mathematical problem is written as

$$
\begin{align*}
u^{\prime}(t) &= f(t,u(t)),\\
u(t_0) &= u_0 ,
\end{align*}
$$

and is commonly referred to as an *initial value problem*, or simply an IVP. All the ODE problems we will consider 
in this book are initial value problems. As an example, consider the very simple ODE

$$
u'=u .
$$

This equation has the general solution $u=Ce^t$ for any constant $C$, so it has an infinite number of solutions.
Specifying an initial condition $u(t_0)=u_0$ gives $C=u_0$, and we get the unique solution $u=u_0e^t$.  
We shall see that, when solving the equation numerically, we need to define $u_0$ in order to start our method and
compute a solution at all.

 === A simple and general solver; the Forward Euler method. ===
A numerical method for ([1](#ode0)) can be derived by simply approximating the derivative
in the equation $u'=f(t,u)$ by a finite difference. To introduce the idea, assume that we have 
already computed $u$ at discrete time points $t_0,t_1,\ldots,t_n$. At time $t_n$ we have the ODE

$$
u'(t_n) = f(t_n,u(t_n)),
$$

and we can now approximate $u'(t_n)$ by a forward finite difference;

$$
u'(t_n)\approx \frac{u(t_{n+1})-u(t_n)}{\Delta t} .
$$

Inserting this approximation into the ODE at $t=t_n$ yields the following equation

$$
\frac{u(t_{n+1})-u(t_n)}{\Delta t} = f(t_n,u(t_n)),
$$

and we can rearrange the terms to obtain an explicit formula for $u(t_{n+1})$:

$$
u(t_{n+1}) = u(t_n) + \Delta t f(t_n,u(t_n)) .
$$

This method is known as the *Forward Euler (FE) method* or the *Explicit Euler Method*,
and is the simplest numerical method for solving an ODE. The classification as a *forward*
or *explicit* method refer to the fact that we have an explicit update formula for $u(t_{n+1})$,
which only involves known quantities at time $t_n$. In contrast, for an *implicit* ODE solver
the update formula will include terms on the form $f(t_{n+1},u(t_{n+1})$, and we need to solve 
a generally nonlinear equation to determine the unknown $u(t_{n+1})$. We will visit other explicit ODE solvers in
Chapter 2 and implicit solvers in Chapter 3. 

To simplify the formula a bit we introduce the notation $u_{n} = u(t_{n})$, i.e., 
we let $u_n$ denote the numerical approximation to the exact solution $u(t)$ at $t=t_n$.
The update formula now reads

<!-- Equation labels as ordinary links -->
<div id="forw_euler"></div>

$$
\begin{equation} u_{n+1} = u_n + \Delta t f(t_n,u_n) ,
\label{forw_euler} \tag{2}
\end{equation}
$$

which, if we know the $u_0$ at time $t_0$, can be applied repeatedly to $u_1$, $u_2$, $u_3$ and so forth.
If we again consider the very simple ODE given by $u' = u$, we have

$$
\begin{align*}
u_1 &= u_0 + \Delta t u_0, \\
u_2 &= u_1 + \Delta t u_1, \\
u_3 &= u_2 + \ldots .
\end{align*}
$$

In a Python program, this repeated application
of the same formula is conveniently implemented in a for-loop, and the solution can 
be stored in a list or a NumPy array. See, for instance, [[sundnes2020introduction]](#sundnes2020introduction) for an introduction to
NumPy arrays and tools, which will be used extensively through these notes. 
For a given final time $T$ and number of time steps $N$, 
we perform the following steps:
1. Create arrays `t` and `u` of length $N+1$

2. Set initial condition: `u[0]` $= u_0$, `t[0]` $= 0$

3. Compute the time step $\Delta t$ (`dt`) $= T/N$

4. For $n=0,1,2,\ldots,N-1$:

  * `t[n+1] = t[n] + dt`

  * `u[n+1] = (1 + dt)*u[n]`


A complete Python implementation of this algorithm may look like

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

N = 20
T = 4
dt = T/N
u0 = 1

t = np.zeros(N+1)
u = np.zeros(N+1)

u[0] = u0
for n in range(N):
    t[n+1] = t[n] + dt
    u[n+1] = (1 + dt)*u[n]

plt.plot(t,u)
plt.show()

Notice that there is no need to set `t[0]= 0` when `t` is created in this way, but updating `u[0]` is important. In fact, 
forgetting to do so is a very common error in ODE programming, so it is worth taking note of the line `u[0] = u0`
The solution is shown in [Figure](#fig:ode0), for two different choices of the time step $\Delta t$. We see that the
approximate solution improves as $\Delta t$ is reduced, although both the solutions are quite inaccurate. However, reducing the
time step further will easily create a solution that cannot be distinguished from the exact solution.

<!-- dom:FIGURE: [./figs_ch1/FE_n_10_20.png, width=600 frac=1] Solution of $u' = u, u(0) = 1$ with $\Delta t = 0.4 (N=10)$ and $\Delta t=0.2 (N=20)$. <div id="fig:ode0"></div> -->
<!-- begin figure -->
<div id="fig:ode0"></div>

<p>Solution of $u' = u, u(0) = 1$ with $\Delta t = 0.4 (N=10)$ and $\Delta t=0.2 (N=20)$.</p>
<img src="./figs_ch1/FE_n_10_20.png" width=600>

<!-- end figure -->


### Extending the solver to the general ODE.

As stated above, the purpose of this chapter is to create general-purpose ODE solvers, that can solve any ODE written
on the form $u'=f(t,u)$. This requires a very small modification of the algorithm above;
1. Create arrays `t` and `u` of length $N+1$

2. Set initial condition: `u[0]` = $u_0$, `t[0]=0`

3. For $n=0,1,2,\ldots,N-1$:

  * `t[n+1] = t[n] + dt`

  * `u[n+1] = u[n] + dt*f(t[n], u[n])`


The only change of the algorithm is in the formula for computing `u[n+1]` from `u[n]`.
In the previous case we had $f(t,u) = u$, and to create a
general-purpose ODE solver we simply replace `u[n]` with the more general `f(t[n],u[n])`.
The following Python function implements this generic version of the FE method:

In [2]:
def forward_euler(f, u0, T, N):
    """Solve u'=f(t,u), u(0)=u0, with n steps until t=T."""
    import numpy as np
    t = np.zeros(N+1)
    u = np.zeros(N+1)  # u[n] is the solution at time t[n]

    u[0] = u0
    dt = T/N

    for n in range(N):
        t[n+1] = t[n] + dt
        u[n+1] = u[n] + dt*f(t[n], u[n])

    return t, u

This simple function can solve any ODE written on the form ([1](#ode0)). The right-hand 
side function $f(t,u)$ needs to be implemented as a Python function, which is
then passed as an argument to `forward_euler` together with the initial condition `u0`, the
stop time `T` and the number of time steps `N`. The two latter arguments are then used
to calculate the time step `dt` inside the function.

To illustrate how the function is used, let us apply it to solve the same problem as above;
$u'=u$, $u(0)=1$, for $t\in [0,4]$. The following code uses the `forward_euler` function to solve this problem:

In [3]:
def f(t, u):
    return u

u0 = 1
T = 3
N = 30
t, u = forward_euler(f, u0, T, N)

The `forward_euler` function returns the two arrays `u` and `t`, which we can
plot or process further as we want. One thing worth noticing in this code is the definition of the
right-hand side function `f`. As we mentioned above, this function should always be written
with two arguments `u` and `t`, although in this case only `u` is used inside the function.
The two arguments are needed because we want our solver to work for all ODEs on the
form $u' = f(t,u)$, and the function is therefore called as `f(t[n], u[n])` inside
the `forward_euler` function. If our right
hand side function was defined as a function of `u` only, i.e., using `def f(u):`, 
we would get an error message when the function was called inside `forward_euler`. 
This problem is solved by simply writing `def f(t,u):` even if `t` is never used 
inside the function.


For being only 15 lines of Python code, the capabilities of the `forward_euler`
function above are quite remarkable. Using this function, we can solve any
kind of linear or nonlinear ODE, most of which would be impossible to solve
using analytical techniques. The general recipe for using this function can be summarized
as follows:
1. Identify $f(t,u)$ in your ODE

2. Make sure you have an initial condition $u_0$

3. Implement the $f(t,u)$ formula in a Python function `f(t, u)`

4. Choose the number of time steps `N`

5. Call `t, u = forward_euler(f, u0, T, N)`

6. Plot the solution

It is worth mentioning that the FE method is the simplest of all
ODE solvers, and many will argue that it is not very good. This is partly true,
since there are many other methods that are more accurate and more stable when applied to
challenging ODEs. We shall look at a few examples of such methods later in this
book. However, the FE method is quite suitable for solving most ODEs.
If we are not happy with the accuracy we can simply reduce the time step, and
in most cases this will give the accuracy we need with a negligible increase in computing time.


# The ODE solver implemented as a class
<div id="sec:fe_class"></div> 
We can increase the flexibility of the `forward_euler`
solver function by implementing it as a class. There are many ways to implement
and such a class, but one possible usage can be as follows:

```Python
        method = ForwardEuler_v0(f) 
        method.set_initial_condition(u0)
        t, u = method.solve(t_span=(0,10),N=100)
        plot(t, u)
```

The benefits of using a class instead of a function may not be obvious at this
point, but it will become clear when we introduce different ODE solvers later. 
For now, let us just look at how such a solver class would need to be implemented in 
order to support the use case specified above:
  * We need a constructor (`__init__`) which takes a single argument, the right-hand side function `f`, 
    and stores it as an attribute.

  * The method `set_initial_condition` takes the initial condition as argument and stores it. 

  * The class needs a `solve`-method, which takes the time interval `t_span` and number of time steps `N`
    as arguments. The method implements the for-loop for solving the ODE and returns
    the solution, similar to the `forward_euler` function considered earlier.

  * The time step $\Delta t$ and the sequences $t_n$, $u_n$ must be
    initialized in one of the methods, and it may also be convenient to store these as attributes. 
    Since the time interval and the number of steps are arguments to the `solve` method it is natural
    to do these computations there.

In addition to these methods, it may be convenient to implement the formula for
advancing the solution one step as a separate method `advance`. In this way it
becomes very easy to implement new numerical methods, since we typically only
need to change the `advance` method. A first version of the solver class may
look as follows:

In [4]:
import numpy as np

class ForwardEuler_v0:
    def __init__(self, f): 
        self.f = f 
        
    def set_initial_condition(self,u0):
        self.u0 = u0

    def solve(self,t_span,N):
        """Compute solution for 
        t_span[0] <= t <= t_span[1],
        using N steps."""
        t0,T = t_span
        self.dt = T/N
        self.t = np.zeros(N+1) 
        self.u = np.zeros(N+1)
        
        self.t[0] = t0
        self.u[0] = self.u0
        
        for n in range(N):
            self.n = n
            self.t[n+1] = self.t[n] + self.dt
            self.u[n+1] = self.advance()
        return self.t, self.u

    def advance(self):
        """Advance the solution one time step."""
        u, dt, f, n, t = self.u, self.dt, self.f, self.n, self.t

        unew = u[n] + dt*f(t[n],u[n])
        return unew

This class does essentially the same tasks as the `forward_euler` function above,
and the main advantage of the class implementation is
the increased flexibility that comes with the `advance` method. As we shall see
later, implementing a different numerical method typically only requires
implementing a new version of this method, while all other code can be left unchanged.

We can also use a class to hold the right-hand side $f(t,u)$, which is
particularly convenient for functions with parameters.
Consider for instance the model for logistic growth;

$$
u^{\prime}(t)=\alpha u(t)\left(  1-\frac{u(t)}{R}\right),\quad u(0)=u_0,\quad t\in [0,40],
$$

which is typically used to model self-limiting growth of a biological population, i.e., growth 
that is constrained by limited resources. The initial growth is approximately exponential, with
growth rate $\alpha$, and the population curve flattens out as the population size approaches the *carrying capacity* $R$,
see [Figure](#fig:logistic) for an example solution. 
The right hand side function includes two parameters $\alpha$ and $R$, but if we
want to solve it using our FE function or class, it must be implemented
as a function of $t$ and $u$ only. There are several ways to do this in Python, but one
convenient approach is to implement the function as a class with a call method. 
We can then define the parameters as attributes in the constructor and
use them inside the `__call__` method:

In [5]:
class Logistic:
    def __init__(self, alpha, R, u0):
        self.alpha = alpha
        self.R = R 
        self.u0 = u0

    def __call__(self, t, u):   # f(t,u)
        return self.alpha*u*(1 - u/self.R)

The main program for solving the logistic growth problem may now look like:

In [6]:
problem = Logistic(alpha=0.2, R=1.0, u0=0.1)
solver = ForwardEuler_v0(problem)
solver.set_initial_condition(problem.u0)
    
T = 40
t, u = solver.solve(t_span=(0,T),N=400)

<!-- dom:FIGURE: [./figs_ch1/logistic_func_mpl.png, width=600 frac=1.0] Solution of the logistic growth model. <div id="fig:logistic"></div> -->
<!-- begin figure -->
<div id="fig:logistic"></div>

<p>Solution of the logistic growth model.</p>
<img src="./figs_ch1/logistic_func_mpl.png" width=600>

<!-- end figure -->

# Systems of ODEs
<div id="sec:ode_sys"></div>
 So far we have only considered ODEs with a single solution component, often called scalar ODEs.
Many interesting processes can be described
by systems of ODEs, i.e., multiple ODEs where the right-hand side of one equation depends on the solution of the others. Such equation
systems are also referred to as vector ODEs. One simple example is

$$
\begin{alignat*}{2}
u' &= v, \quad &&u(0) = 1\\
v' &= -u, \quad &&v(0) = 0.
\end{alignat*}
$$

The solution of this system is $u=\cos t, v=\sin t$, which can easily be verified by inserting the solution into the equations
and the initial conditions. For more general cases, it is usually even more difficult to find analytical solutions of ODE systems
than of scalar ODEs, and numerical methods are usually required. In this section we will extend the solvers introduced in
the sections [Creating a general-purpose ODE solver](#sec:fe_intro)-[The ODE solver implemented as a class](#sec:fe_class) to be able to solve systems of ODEs. We shall see that such an 
extension requires relatively small modifications of the code.

We want to develop general software that can be applied to any vector ODE or scalar ODE, and for this purpose it is
useful to introduce some general mathematical notation. We have $n$ unknowns

$$
u^{(0)}(t), u^{(1)}(t), \ldots, u^{(n-1)}(t)
$$

in a system of $n$ ODEs:

$$
\begin{align*}
{d\over dt}u^{(0)} &= f^{(0)}(t, u^{(0)}, u^{(1)}, \ldots, u^{(n-1)}),\\
{d\over dt}u^{(1)} &= f^{(1)}(t, u^{(0)}, u^{(1)}, \ldots, u^{(n-1)}),\\
\vdots &= \vdots\\
{d\over dt}u^{(n-1)} &= f^{(n-1)}(t, u^{(0)}, u^{(1)}, \ldots, u^{(n-1)}).
\end{align*}
$$

To simplify the notation (and later the implementation), we collect both the solutions $u^{(i)}(t)$
and right-hand side functions $f^{(i)}$ into vectors;

$$
u = (u^{(0)}, u^{(1)}, \ldots, u^{(n-1)}),
$$

and

$$
f = (f^{(0)}, f^{(1)}, \ldots, f^{(n-1)}).
$$

Note that $f$ is now a vector-valued function. It takes $n+1$ input arguments ($t$ and the $n$ components of $u$) and returns
a vector of $n$ values.
Using this notation, the ODE system can be written

$$
u' = f(t, u),\quad u(t_0) = u_0,
$$

where $u$ and $f$ are now vectors and $u_0$ is a vector of initial conditions. We see that we use exactly the
same notation as for scalar ODEs, and whether we solve a scalar or system of ODEs is determined by how we define $f$ and the initial
condition $u_0$. This general notation is completely standard in text books on ODEs, and we can easily make the Python
implementation just as general. The generalization of our ODE solvers is facilitated considerably by the convenience of NumPy
arrays and vectorized computations.

# A `ForwardEuler` class for systems of ODEs
The `ForwardEuler_v0` class above was written for scalar ODEs, and we now want to make it work for a system
$u'=f$, $u(0)=u_0$, where $u$, $f$ and $u_0$ are vectors (arrays). To identify how the code needs to be changed, 
let us first revisit the underlying numerical method. Using the general notation introduced above, 
applying the forward Euler method to a system of ODEs yields an update formula that looks exactly 
as for the scalar case, but where all the terms are vectors:

$$
\underbrace{u_{k+1}}_{\mbox{vector}} =
\underbrace{u_k}_{\mbox{vector}} +
\Delta t\, \underbrace{f(u_k, t_k)}_{\mbox{vector}} .
$$

We could also write this formula in terms of the individual components, as in

$$
u^{(i)}_{k+1} = u^{(i)}_{k} + \Delta t f^{(i)}(t_k, u_{k}), \mbox{ for } i = 0,\ldots , {n-1},
$$

but the compact vector notation is much easier to read. Fortunately, the way we write the vector
version of the formula is also how NumPy arrays are used in calculations. The
Python code for the formula above may therefore look identical to the version for scalar ODEs;

In [7]:
u[k+1] = u[k] + dt*f(t[k], u[k])

with the important difference that both `u[k]` and `u[k+1]` are now arrays.
Since these are arrays, the solution `u` must be a
two-dimensional array, and `u[k],u[k+1]`, etc. are the rows of this array.
The function `f` expects an array as its second argument, and must return a one-dimensional array,
containing all the right-hand sides $f^{(0)},\ldots,f^{(n-1)}$. To get a better
feel for how these arrays look and how they are used,
we may compare the array holding the solution of a scalar ODE to that of a system of two ODEs.
For the scalar equation, both `t` and `u` are one-dimensional NumPy
arrays, and indexing into `u` gives us numbers, representing the solution at each time step.
For instance, we may have

        t = array([0. ,  0.4, 0.8, 1.2, (...) ])
        
        u = array([1. , 1.4, 1.96, 2.744, (...)])
        
        u[0] = 1.0
        u[1] = 1.4
        
        (...)


In the case of a system of two ODEs, `t` is still a one-dimensional array, but the solution array `u` is
now two-dimensional, with one column for each solution component. We can index it exactly as shown above, 
and the result is a one-dimensional array of length two, which holds the two solution components
at a single time step:

        u = array([[1.0, 0.8],
                   [1.4, 1.1],
                   [1.9, 2.7],
                      (...)])
        
        u[0] = array([1.0, 0.8])
        u[1] = array([1.4, 1.1])
        
        (...)


The similarity of the generic notation for vector and scalar ODEs, and the
convenient algebra of NumPy arrays, indicate that the solver
implementation for scalar and system ODEs can also be very similar. This is indeed true,
and the `ForwardEuler_v0` class from the previous chapter can be made to work for ODE
systems by a few minor modifications:
 * Ensure that `f(t,u)` always returns an array.

 * Inspect $u_0$ to see if it is a single number or a list/array/tuple and
   make the `u` either a one-dimensional or two-dimensional  array. 

If these two items are handled and initialized correctly, the rest of the code from
the section [The ODE solver implemented as a class](#sec:fe_class) will in fact work with no modifications.

: This step is not strictly needed, since we could use a two-dimensional array
with size `(N+1,1)` for scalar ODEs. However, using a one-dimensional array for scalar
ODEs gives simpler and more intuitive indexing. 

The extended class implementation may look like:

In [8]:
import numpy as np

class ForwardEuler:
    def __init__(self, f): 
        self.f = lambda t,u: np.asarray(f(t,u),float)

    
    def set_initial_condition(self,u0):
        if isinstance(u0, (float,int)):  #scalar ODE
            self.neq = 1                 #number of eqs.
            u0 = float(u0)
        else:
            u0 = np.asarray(u0)
            self.neq = u0.size
        self.u0 = u0
    
    def solve(self,t_span,N):
        """Compute solution for 
        t_span[0] <= t <= t_span[1],
        using N steps."""
        t0,T = t_span
        self.dt = (T-t0)/N
        self.t = np.zeros(N+1)
        if self.neq == 1:
            self.u = np.zeros(N+1)
        else:
            self.u = np.zeros((N+1,self.neq))
        
        self.t[0] = t0
        self.u[0] = self.u0
    
        for n in range(N):
            self.n = n
            self.t[n+1] = self.t[n] + self.dt
            self.u[n+1] = self.advance()
        return self.t, self.u

    def advance(self):
        """Advance the solution one time step."""
        u, dt, f, n, t = self.u, self.dt, self.f, self.n, self.t

        unew = u[n] + dt*f(t[n],u[n])
        return unew

It is worth commenting on some parts of this code. First, the constructor looks
almost identical to the scalar case, but we use a lambda function and
the convenient `np.asarray` function to convert any `f` that 
returns a list or tuple to a function returning a NumPy array. If `f` already returns an
array, `np.asarray` will simply return this array with no changes. This modification is not strictly
needed, since we could just assume that the user implements `f` to return
an array, but it makes the class more robust and flexible. We have also included
a couple of tests in the `set_initial_condition` method, to check if `u0` is a single
number (`float`) or a NumPy array, and define the attribute `self.neq` to
hold the number of equations.
The final modification is found in the method `solve`, where
the `self.neq` attribute is inspected and `u` is
initialized to a one- or two-dimensional array of the correct size. The
actual for-loop and the `advance` method are both identical to the previous version of the class.

### Example: ODE model for a pendulum.

To demonstrate the use of
the updated `ForwardEuler` class, we consider a system of ODEs describing the motion 
of a simple pendulum, as illustrated in [Figure](#fig:pendulum). This nonlinear system
is a classic physics problem, and despite its simplicity it is not possible to find 
an exact analytical solution. We will formulate the system in terms of two main
variables; the angle $\theta$ and the angular velocity $\omega$, see [Figure](#fig:pendulum).
For a simple pendulum with no friction, the dynamics of these two variables is governed by

<!-- Equation labels as ordinary links -->
<div id="pendulum1"></div>

$$
\begin{equation}
\frac{d \theta}{dt} = \omega, \label{pendulum1} \tag{3} 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="pendulum2"></div>

$$
\begin{equation} 
\frac{d \omega}{dt} = -\frac{g}{L}\sin(\theta), \label{pendulum2} \tag{4}  
\end{equation}
$$

where $L$ is the length of the pendulum and $g$ is the gravitational constant. Eq. ([3](#pendulum1)) follows
directly from the definition of the angular velocity, while ([4](#pendulum2)) follows from Newton's second 
law, where $d\omega/dt$ is the acceleration and the right-hand side is the tangential component of
the gravitational force acting on the pendulum, divided by its mass.  
To solve the system we need to define initial conditions for both unknowns,
i.e., we need to know the initial position and velocity of the pendulum. 

<!-- dom:FIGURE: [./figs_ch1/pendulum.png, width=100 frac=0.2] Illustration of the pendulum problem. The main variables of interest are the angle $\theta$ and its derivative $\omega$ (the angular velocity). <div id="fig:pendulum"></div> -->
<!-- begin figure -->
<div id="fig:pendulum"></div>

<p>Illustration of the pendulum problem. The main variables of interest are the angle $\theta$ and its derivative $\omega$ (the angular velocity).</p>
<img src="./figs_ch1/pendulum.png" width=100>

<!-- end figure -->


Since the right-hand side defined by ([3](#pendulum1))-([4](#pendulum2)) includes the 
parameters $L$ and $g$, it is convenient to implement it as a class, as illustrated for the logistic
growth model earlier. A possible implementation may look like this:

In [9]:
from math import sin

class Pendulum:
    def __init__(self, L, g = 9.81):
        self.L = L 
        self.g = g 
    
    def __call__(self, t, u):
        theta, omega = u
        dtheta = omega
        domega = -self.g/self.L * sin(theta)
        return [dtheta, domega]

We see that the function returns a list, but this will automatically
be wrapped into a function returning an array by the solver class' constructor,
as mentioned above. The main program is not very different
from the examples of the previous chapter, except that we need to define an
initial condition with two components. Assuming that this class definition as 
well as the `ForwardEuler` exist in the same file, the code to solve the pendulum problem
can look like this:

In [10]:
import matplotlib.pyplot as plt

problem = Pendulum(L=1)
solver = ForwardEuler(problem)
theta0, omega0 = np.pi/4, 0
solver.set_initial_condition((theta0,omega0))
T = 10
N = 1000
t, u = solver.solve(t_span=(0,T),N=N)

plt.plot(t,u[:,0],label=r'$\theta$')
plt.plot(t,u[:,1],label=r'$\omega$')
plt.legend()
plt.show()

Notice that since `u` is a two-dimensional array, we use array slicing to extract and plot the individual components.
In this specific example a call like `plt.plot(t,u)` would also work, and would plot both solution
components. However, we are often interested in plotting selected components of the solution, and
in this case the array slicing is needed. The resulting plot is shown in [Figure](#fig:pendulum_sol). 
Another minor detail worth noticing is use of Python's raw string format for the labels, indicated
by the `r` in front of the string. Raw strings will treat the backslash (`\`) as a regular character, and is
often needed when using Latex encoding of mathematical symbols. 
The observant reader may also notice that the amplitude of the pendulum motion appears to increase over time, 
which is clearly not physically correct. In fact, for an undamped pendulum problem defined by 
([3](#pendulum1))-([4](#pendulum2)), the energy is conserved, and the amplitude should therefore be constant. 
The increasing amplitude is a numerical artefact introduced by the forward Euler method, and the solution
may be improved by reducing the time step or replacing the numerical method. 

<!-- dom:FIGURE: [./figs_ch1/pendulum_FE.png, width=600 frac=1] Solution of the simple pendulum problem, computed with the forward Euler method. <div id="fig:pendulum_sol"></div> -->
<!-- begin figure -->
<div id="fig:pendulum_sol"></div>

<p>Solution of the simple pendulum problem, computed with the forward Euler method.</p>
<img src="./figs_ch1/pendulum_FE.png" width=600>

<!-- end figure -->



# Checking the error in the numerical solution
<div id="sec:error_taylor"></div>

Recall from the section [Creating a general-purpose ODE solver](#sec:fe_intro) that we derived the FE method by approximating the
derivative with a finite difference;

<!-- Equation labels as ordinary links -->
<div id="fd_approx"></div>

$$
\begin{equation} 
    u'(t_n)\approx \frac{u(t_{n+1})-u(t_n)}{\Delta t} .
\label{fd_approx} \tag{5}\end{equation}
$$

This approximation obviously introduces an error, and since we approach the true derivative as $\Delta t\rightarrow 0$ it
is quite intuitive that the error depends on the size of the time step $\Delta t$. This relation was demonstrated
visually in [Figure](#fig:ode0), but it would of course be valuable to have a more precise quantification of how the
error depends on the time step. Analyzing the error in numerical methods is a large branch of applied mathematics, 
which we will not cover in detail here, and the interested reader is referred to, for instance [[ODEI]](#ODEI). 
However, when implementing a numerical method it is very useful to know its theoretical accuracy, and
in particular to be able to compute the error and verify that it behaves as expected.

The Taylor expansion, also discussed briefly in Appendix A, is an essential tool for 
estimating the error in numerical methods for ODEs. For any smooth function $\hat{u}(t)$, if we 
can compute the function value and its derivatives at $t_n$, the value at $t_n+\Delta t$ 
can be approximated by the series

$$
\hat{u}(t_n+\Delta t) = \hat{u}(t_n) + \Delta t \hat{u}'(t_n) +\frac{\Delta t^2}{2}\hat{u}''(t_n) + 
    \frac{\Delta t^3}{6}\hat{u}'''(t_n)+O(\Delta t^4).
$$

We can include as many terms as we like, and since $\Delta t$ is small we always have $\Delta t^(n+1) \ll \Delta t^n$, so
the error in the approximation is dominated by the first neglected term.  
The update formula of the FE method, derived from ([5](#fd_approx)), was

$$
u_{n+1} = u(t_n) + \Delta t u'(t_n),
$$

which we may recognize as a Taylor series truncated after the first order term, and we expect the error
$|u_{n+1}- \hat{u}_{n+1}|$ to be proportional to $\Delta x^2$. Since this is the error for a single time step, the
accumulated error after $N\sim 1/\Delta t $ steps is proportional to $\Delta t$, and the 
FE method is hence a \emph{first order} method. As we will see in Chapter 
2, more accurate methods can be constructed by deriving update formulas that make more
terms in the Taylor expansion of the error cancel. This process is fairly straightforward for low-order methods, 
e.g., of second or third order, but it quickly gets complicated for high order solvers, see, for 
instance [[ODEI]](#ODEI) for details. 

Knowing the theoretical accuracy of an ODE solver is important for a number of reasons, and one of them is that it
provides a method for verifying our implementation of the solver. If we can solve a given problem and demonstrate that
the error behaves as predicted by the theory, it gives a good indication that the solver is implemented correctly. 
We can illustrate this procedure using the simple initial value problem introduced earlier;

$$
u' = u, \quad u(0) = 1 .
$$

As stated above, this problem has the analytical solution $u=e^t$, and we can use this to compute the 
error in our numerical solution. But how should the error be defined?  
There is no unique answer to this question. For practical applications,
the so-called root-mean-square (RMS) or relative-root-mean-square (RRMS) are commonly used error measures, defined by

$$
\begin{align*}
RMSE &=  \sqrt{\frac{1}{N}\sum_{n=0}^N(u_n-\hat{u}(t_n))^2}, \\
RRMSE &= \sqrt{\frac{1}{N}\sum_{n=0}^N\frac{(u_n-\hat{u}(t_n))^2}{\hat{u}(t_n)^2}}, 
\end{align*}
$$

respectively. Here, $u_n$ is the numerical solution at time step $n$ and $\hat{u}(t_n)$ the corresponding
exact solution. In more mathematically oriented texts, the errors are usually defined in terms of \emph{norms}, for 
instance the discrete $l_1,l_2,$ and $l_\infty$ norms:

$$
\begin{align*}
 e_{l_1} &= \sum_{i=0}^N(|u_i-\hat{u}(t_i)|),\\
 e_{l_2} &= \sum_{i=0}^N(u_i-\hat{u}(t_i)^2), \\
 e_{l_\infty} &= \max_{i=0}^N(\hat{u}_i-u(t_i)).  
\end{align*}
$$

While the choice of error norm may be important for certain cases, for practical
applications it is usually not very important, and all the different error measures can be expected to behave
as predicted by the theory. For simplicity, we will apply an even simpler error measure for our example, where
we simply compute the error at the final time $T$, given by $e = |u_N-\hat{u}(t_N)|$. Using the `ForwardEuler`
class introduced above, the complete code for checking the convergence may look as follows:

In [11]:
from forward_euler_class_v1 import *
import numpy as np

def rhs(t,u):
    return u

def exact(t):
    return np.exp(t)

solver = ForwardEuler(rhs)
solver.set_initial_condition(1.0)

T = 3.0
t_span = (0,T)
N = 30

print('Time step (dt)   Error (e)      e/dt')
for _ in range(10):
    t,u = solver.solve(t_span,N)
    dt = T/N
    e = abs(u[-1]-exact(T))
    print(f'{dt:<14.7f}   {e:<12.7f}   {e/dt:5.4f}')
    N = N*2

This program will produce the following output when it is run in the terminal:

        Time step (dt)   Error (e)      e/dt
        0.1000000        2.6361347      26.3613
        0.0500000        1.4063510      28.1270
        0.0250000        0.7273871      29.0955
        0.0125000        0.3700434      29.6035
        0.0062500        0.1866483      29.8637
        0.0031250        0.0937359      29.9955
        0.0015625        0.0469715      30.0618
        0.0007813        0.0235117      30.0950
        0.0003906        0.0117624      30.1116
        0.0001953        0.0058828      30.1200


In the rightmost column we see that the error divided by the time step is approximately constant, which
supports the theoretical result that the error is proportional to $\Delta t$. In subsequent chapters we will
perform similar calculations for higher order methods, to confirm that the error is proportional to $\Delta t^r$,
where $r$ is the theoretical order of convergence for the method. 

In order to compute the error in our numerical solution we obviously need to know the true solution to our 
initial value problem. This part was easy for the simple example above, since we knew the analytical solution
to the equation, but this solution is only available for very simple ODE problems. It is, however, often 
interesting to estimate the error and the order of convergence for more complex problems, and for this task we 
need to take a different approach. Several alternatives exist, including for instance the *method of manufactured
solutions*, where one simply choses a solution function $u(t)$ and computes its derivative analytically to 
determine the right-hand side of the ODE. An even simpler approach, which usually works well, is to 
compute a very accurate numerical solution using a high-order solver and small time steps, and use this
solution as the reference for computing the error. For accurate error estimates it is of course essential
that the reference solution is considerably more accurate than the numerical solution we want to evaluate. 
The reference solution therefore typically requires very small time steps and can take some time to 
compute, but in most cases the computation time will not be a problem. 












# Using ODE solvers from SciPy
 As mentioned in the preface to this book, there are numerous ODE solvers around that can 
be used directly, so one may argue that there is no need to implement our own solvers. 
This may indeed be true, but, as we have argued earlier, it is sometimes very useful to 
know the inner workings of the solvers we apply, and the best way to obtain this 
knowledge is to implement the solvers ourselves. However, if we have a given ODE model and
want to solve it as efficiently as possible, there are several existing solvers to choose from.
For Python programmers, the most natural choice may be the solvers from *SciPy*, which 
have evolved into a robust and fairly efficient suite of ODE solvers. SciPy is a large suite 
of scientific software in Python, including tools for linear algebra, optimization, integration,
and other common tasks of scientific computing. For solving initial 
value problems, the tool of choice is the `solve_ivp` function from the `integrate` module. 
The following code applies `solve_ivp` with the `Pendulum` class presented above to solve the
simple pendulum problem defined by ([3](#pendulum1))-([4](#pendulum2)). We assume that the `Pendulum`
class is saved in a separate file `pendulum.py`.

In [12]:
from scipy.integrate import solve_ivp
import numpy as np
import matplotlib.pyplot as plt
from pendulum import Pendulum

problem = Pendulum(L=1)
t_span = (0,10.0)
u0 = (np.pi/4, 0)

solution = solve_ivp(problem, t_span, u0)

plt.plot(solution.t, solution.y[0,:])
plt.plot(solution.t, solution.y[1,:])
plt.legend([r'$\theta$',r'$\omega$'])
plt.show()

<!-- dom:FIGURE: [./figs_ch1/pendulum_scipy.png, width=600 frac=1] Solution of the simple pendulum problem, computed with the SciPy `solve_ivp` function and the default tolerance. <div id="fig:pendulum_scipy"></div> -->
<!-- begin figure -->
<div id="fig:pendulum_scipy"></div>

<p>Solution of the simple pendulum problem, computed with the SciPy <code>solve_ivp</code> function and the default tolerance.</p>
<img src="./figs_ch1/pendulum_scipy.png" width=600>

<!-- end figure -->


Running this code will result in a plot similar to [Figure](#fig:pendulum_scipy), and we observe that
the solution does not look nearly as nice as the one we got from the `ForwardEuler` solver above. 
The reason for this apparent error is that `solve_ivp` is an *adaptive* solver, which chooses the 
time step automatically to satisfy a given error tolerance. The default value of this tolerance is relatively large, 
which results in the solver using very few time steps and the solution plots looking edgy. 
If we compare the plot with the exact solution, indicated by the two dotted curves
[Figure](#fig:pendulum_scipy), we see that the solution at the time points $t_n$ 
is quite accurate, but the linear interpolation between the time points completely destroys the visual 
appearance. A more visually appealing solution can be obtained in several ways. 
We may, for instance, pass the function an additional argument `t_eval`, which is a NumPy array 
containing the time points where we want to evaluate the solution:

In [13]:
t_eval = np.linspace(0,10.0,1001)
solution = solve_ivp(problem, t_span, u0,t_eval=t_eval)

Alternatively, we can reduce the error tolerance of the solver, for instance setting

In [14]:
rtol = 1e-6
solution = solve_ivp(problem, t_span, u0,rtol=rtol)

This latter call will reduce the relative tolerance `rtol` from its default value of `1e-3` (0.001). 
We could also adjust the absolute tolerance using the parameter `atol`. While we will not
consider all the possible arguments and options to `solve_ivp` here, we mention that we can also 
change the numerical method used by the function, by passing in a parameter named `method`. For instance, 
a call like

In [15]:
rtol = 1e-6
solution = solve_ivp(problem, t_span, u0,method='Radau')

will replace the default solver (called `rk45`) with an implicit Radau ODE solver, which we will introduce and
explain in Chapter 3. For a complete description of parameters accepted by the `solve_ivp` 
function we refer to the online SciPy documentation.